<a href="https://colab.research.google.com/github/JiaxinLi-lipluszn/Temp_notebooks/blob/main/Dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import 

In [31]:
import os
import sys
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import datasets
from torchvision import transforms as pth_transforms
from torchvision import models as torchvision_models
import pandas as pd
from torch.utils.data import Dataset

# Try the ImageFolder dataloader

## Unzip the FashiMNIST into content folder

In [ ]:
!unzip /content/drive/MyDrive/data.zip -d /content/ 
!mv /content/data/test /content/data/val

In [5]:
class ReturnIndexDataset(datasets.ImageFolder):
    def __getitem__(self, idx):
        img, lab = super(ReturnIndexDataset, self).__getitem__(idx)
        return img, idx

In [6]:
transform = pth_transforms.Compose([
        pth_transforms.Resize(256, interpolation=3),
        pth_transforms.CenterCrop(224),
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
dataset_train = ReturnIndexDataset("/content/data/train/", transform=transform)
dataset_val = ReturnIndexDataset("/content/data/val/", transform=transform)
   

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [15]:
x, y = dataset_train[1000]

In [16]:
y

1000

In [11]:
dataset_val.samples[9999]

('/content/data/val/9/9995.png', 9)

# My Implementation

In [17]:
class DINO_Crop(object):
  def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
    self.global_crops_scale = global_crops_scale
    self.local_crops_scale = local_crops_scale
    self.local_crops_number = local_crops_number
    
  def __call__(self, x):
    inputs = []
    data = x  
    #print(data.shape)
    length = x.shape[0]
    
    global_size = int(length * self.global_crops_scale)
    #local_size = int(length * self.local_crops_scale)
    global_index_1 = np.random.choice(length, size = global_size, replace = False)
    #global_index_2 = np.random.choice(length, size = global_size, replace = False)
    
    global_corr_1 = torch.from_numpy(global_index_1)
    #global_corr_2 = torch.from_numpy(global_index_2)
    
    global_data_1 = data[global_index_1,]
    #global_data_2 = data[global_index_2,]
    #print(data.shape)
    #print(corr.shape)
    #labels = torch.from_numpy(label.item()*np.ones(shape = (784,1))).float()
    global_input_1 = torch.cat([global_data_1, global_corr_1]).float()
    #global_input_2 = torch.cat([global_data_2, global_corr_2]).float()
    #nputs.append(global_input_1)
    #inputs.append(global_input_2)
    '''
    for i in range(self.local_crops_number):
      index = np.random.choice(length, size = local_size, replace = False)
      corr = torch.from_numpy(index)
      data_local = data[index,]
      input_local = torch.cat([data_local, corr]).float()
      inputs.append(input_local)
    '''
    return global_input_1

In [18]:
perturb_list = ["A2M", "CCND1", "DAG1", "FBL", "IDH2", "IFNGR1","JAK1", "MIA", "PIK3IP1", "SLC19A1"]
  

In [23]:
expr = pd.DataFrame()
for label in perturb_list:
      path = "/content/drive/MyDrive/scRNA/subset_hvg/train" + "/" + label + ".csv"
      df = pd.read_csv(path, index_col = 0)
      expr = pd.merge(expr, df, how = "outer", left_index = True, right_index = True)
      

In [28]:
len(list(expr.columns))

4014

In [93]:
class scRNACSV(Dataset):
  def __init__(self, expression_matrix_path, perturb_list, transform = DINO_Crop, target_transform=None):
    self.expr = pd.DataFrame()
    self.samples = []
    self.labels = []
    for label in perturb_list:
      path = expression_matrix_path + "/" + label + ".csv"
      df = pd.read_csv(path, index_col = 0)
      self.expr = pd.merge(self.expr, df, how = "outer", left_index = True, right_index = True)
      self.samples = self.samples + list(self.expr.columns)
      lb = [label for i in range(len(list(self.expr.columns)))]
      self.labels = self.labels + lb
    #self.expr = pd.read_csv(expression_matrix_path, index_col= 0)
    self.transform = transform
    self.target_transform = target_transform
    self.samples = [(self.samples[i],self.labels[i]) for i in range(len(self.labels))]
  def __len__(self):
    return self.expr.shape[1]

  def __getitem__(self,idx):
    #print(self.expr.shape)
    one_cell = torch.from_numpy(np.array(self.expr.iloc[:,idx]))
    #print(f"Here is the shape of one_cell {one_cell.shape}")
    ret = self.transform(one_cell)
    #print(len(ret))
    #print(len(label))
    #lab = 1
    return ret, idx

In [55]:
class scRNACSV(Dataset):
  def __init__(self, expression_matrix_path, perturb_list, transform = DINO_Crop, target_transform=None):
    self.expr = pd.DataFrame()
    self.sample = []
    self.labels = []
    for number_lab, label in zip(range(len(perturb_list)),perturb_list):
      path = expression_matrix_path + "/" + label + ".csv"
      df = pd.read_csv(path, index_col = 0)
      self.expr = pd.merge(self.expr, df, how = "outer", left_index = True, right_index = True)
      self.sample = self.sample + list(self.expr.columns)
      lb = [number_lab for i in range(len(list(df.columns)))]
      self.labels = self.labels + lb
    #self.expr = pd.read_csv(expression_matrix_path, index_col= 0)
    self.transform = transform
    self.target_transform = target_transform
    print(self.expr.shape)
    self.samples = [(self.sample[i], self.labels[i]) for i in range(len(self.labels))]
  def __len__(self):
    return self.expr.shape[1]

  def __getitem__(self,idx):
    #print(self.expr.shape)
    one_cell = torch.from_numpy(np.array(self.expr.iloc[:,idx]))
    #print(f"Here is the shape of one_cell {one_cell.shape}")
    ret = self.transform(one_cell)
    #print(len(ret))
    #print(len(label))
    #lab = 1
    return ret, idx


In [112]:
train = scRNACSV("/content/drive/MyDrive/scRNA/subset_hvg/train", perturb_list, transform=transform)
 

In [107]:
data_loader = torch.utils.data.DataLoader(
        train,
        batch_size= 32,
        shuffle = True
    )

In [113]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

In [111]:
transform = DINO_Crop(0.7,0.25, 8)

In [117]:
train_features, train_labels = next(iter(train_dataloader))

In [118]:
train_features.shape

torch.Size([64, 2800])

In [57]:
len(train.samples)

4014

In [42]:
torch.tensor([s for s in train.samples], dtype = 'str')

TypeError: ignored

In [39]:
train_labels = torch.tensor([s for s in train.samples]).long()


ValueError: ignored

In [36]:
train_labels

tensor([0, 0, 0,  ..., 9, 9, 9])

# Generate PseudoData

In [58]:
!pwd

/content


In [59]:
os.chdir("/content/drive/MyDrive/scRNA/pseudo_data")

In [60]:
!cd test

In [64]:
import numpy as np
np.zeros((2000, 100)).shape

(2000, 100)

In [68]:
import pandas as pd

df = pd.DataFrame(np.zeros((2000,100)), columns= [f"{i}_0" for i in range(100)]) 

In [73]:
3 * np.ones((2000,100))

array([[3., 3., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 3., 3., 3.],
       ...,
       [3., 3., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 3., 3., 3.]])

In [70]:
df.to_csv("./first_test.csv")

In [75]:
for i in range(10):
  df = pd.DataFrame(i * np.ones((2000,400)), columns= [f"{j}_{i}" for j in range(400)]) 
  file_name = f"{i}.csv"
  df.to_csv(file_name)

In [89]:
os.chdir("/content/drive/MyDrive/scRNA/pseudo_data/test")


In [90]:
!pwd

/content/drive/MyDrive/scRNA/pseudo_data/test


In [92]:
for i in range(10):
  df = pd.DataFrame(i * np.ones((2000,100)), columns= [f"{j}_{i}" for j in range(100)]) 
  file_name = f"{i}.csv"
  df.to_csv(file_name)

# Compare the dataloader

In [119]:
class DINO_Crop(object):
  def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
    self.global_crops_scale = global_crops_scale
    self.local_crops_scale = local_crops_scale
    self.local_crops_number = local_crops_number
    
  def __call__(self, x):
    inputs = []
    data = x  
    #print(data.shape)
    length = x.shape[0]
    
    global_size = int(length * self.global_crops_scale)
    local_size = int(length * self.local_crops_scale)
    global_index_1 = np.random.choice(length, size = global_size, replace = False)
    global_index_2 = np.random.choice(length, size = global_size, replace = False)
    
    global_corr_1 = torch.from_numpy(global_index_1)
    global_corr_2 = torch.from_numpy(global_index_2)
    
    global_data_1 = data[global_index_1,]
    global_data_2 = data[global_index_2,]
    #print(data.shape)
    #print(corr.shape)
    #labels = torch.from_numpy(label.item()*np.ones(shape = (784,1))).float()
    global_input_1 = torch.cat([global_data_1, global_corr_1]).float()
    global_input_2 = torch.cat([global_data_2, global_corr_2]).float()
    inputs.append(global_input_1)
    inputs.append(global_input_2)
    for i in range(self.local_crops_number):
      index = np.random.choice(length, size = local_size, replace = False)
      corr = torch.from_numpy(index)
      data_local = data[index,]
      input_local = torch.cat([data_local, corr]).float()
      inputs.append(input_local)
    return inputs


In [120]:
class scRNACSV(Dataset):
  def __init__(self, expression_matrix_path, perturb_list, transform = DINO_Crop, target_transform=None):
    self.expr = pd.DataFrame()
    self.label = []
    for label in perturb_list:
      path = expression_matrix_path + "/" + label + ".csv"
      df = pd.read_csv(path, index_col = 0)
      self.expr = pd.merge(self.expr, df, how = "outer", left_index = True, right_index = True)
      self.label.append(label)

    #self.expr = pd.read_csv(expression_matrix_path, index_col= 0)
    self.transform = transform
    self.target_transform = target_transform
    #print(f"Here is the shape of expr :{self.expr.shape}")
  def __len__(self):
    return self.expr.shape[1]

  def __getitem__(self,idx):
    #print(self.expr.shape)
    one_cell = torch.from_numpy(np.array(self.expr.iloc[:,idx]))
    #print(f"Here is the shape of one_cell {one_cell.shape}")
    ret = self.transform(one_cell)
    
    #print(len(ret))
    #print(len(label))
    lab = 1
    return ret, lab


In [121]:
 Perturb_labels = ["A2M", "CCND1"]

In [122]:
crop = DINO_Crop(0.7, 0.25, 8)

In [182]:
dataset = scRNACSV("/content/drive/MyDrive/scRNA/subset_hvg/train", Perturb_labels, transform=crop)
    

In [159]:
len(dataset)

774

In [160]:
x, y = dataset[0]

In [161]:
y

1

In [162]:
len(x)

10

In [163]:
x[0].shape

torch.Size([2800])

In [164]:
x[9].shape

torch.Size([1000])

In [183]:
data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=64,
        pin_memory=True,
        drop_last=True,
    )

In [184]:
train_features, train_labels = next(iter(data_loader))

In [178]:
len(train_features)

10

In [179]:
train_labels.shape

torch.Size([64])

In [185]:
train_features[0].shape

torch.Size([64, 2800])

# The original dataloder

In [142]:
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path

import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import models as torchvision_models
import numpy as np


In [140]:
class Crop(object):
  def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
    self.global_crops_scale = global_crops_scale
    self.local_crops_scale = local_crops_scale
    self.local_crops_number = local_crops_number
    self.totensor = transforms.ToTensor()
  def __call__(self, x):
    inputs = []
    data = self.totensor(x)
    data = torch.flatten(data)  
    global_size = int(784 * self.global_crops_scale)
    local_size = int(784 * self.local_crops_scale)
    global_index_1 = np.random.choice(784, size = global_size, replace = False)
    global_index_2 = np.random.choice(784, size = global_size, replace = False)
    
    global_corr_1 = torch.from_numpy(global_index_1)
    global_corr_2 = torch.from_numpy(global_index_2)
    
    global_data_1 = data[global_index_1,]
    global_data_2 = data[global_index_2,]
    #print(data.shape)
    #print(corr.shape)
    #labels = torch.from_numpy(label.item()*np.ones(shape = (784,1))).float()
    global_input_1 = torch.cat([global_data_1, global_corr_1]).float()
    global_input_2 = torch.cat([global_data_2, global_corr_2]).float()
    inputs.append(global_input_1)
    inputs.append(global_input_2)
    for i in range(self.local_crops_number):
      index = np.random.choice(784, size = local_size, replace = False)
      corr = torch.from_numpy(index)
      data_local = data[index,]
      input_local = torch.cat([data_local, corr]).float()
      inputs.append(input_local)
    return inputs

In [143]:
o_crop = Crop(0.7, 0.25, 8)

In [170]:
dataset = datasets.FashionMNIST(root = "/content/drive/MyDrive/DINO_Group/Jul_10", download = True, transform = o_crop)
  

In [171]:
len(dataset)

60000

In [172]:
x_o, y_o = dataset[0]

In [173]:
len(x_o)

10

In [174]:
y_o

9

In [175]:
x_o[2].shape

torch.Size([392])